# SemEval 2019 Task A

In [10]:
import pandas as pd
with open('Subtask-A-master\\Subtask-A-master\\V1.4_Training.csv','r',encoding='utf-8') as file:
    df = pd.read_csv(file,names=['No','Text','Label'],delimiter=',')
    df = df.reset_index(drop=True)
    df = df.set_index('No')

In [11]:
df.head()

,Text,Label
No,,
663_3,"""Please enable removing language code from the...",1
663_4,"""Note: in your .csproj file, there is a Suppor...",0
664_1,"""Wich means the new version not fully replaced...",0
664_2,"""Some of my users will still receive the old x...",0
664_3,"""The store randomly gives the old xap or the n...",0


In [15]:
df.Label.unique()

array([1, 0], dtype=int64)

In [26]:
with open('Subtask-A-master\\Subtask-A-master\\SubtaskA_EvaluationData.csv','r',encoding='utf-8') as f:
    df_test = pd.read_csv(f,names=['No','Text','Label'],delimiter=',')
    df_test = df_test.reset_index(drop=True)
    df_test = df_test.set_index('No')

In [27]:
df_test.head()

,Text,Label
No,,
9566,This would enable live traffic aware apps.,X
9569,Please try other formatting like bold italics ...,X
9576,Since computers were invented to save time I s...,X
9577,Allow rearranging if the user wants to change ...,X
9579,Add SIMD instructions for better use of ARM NE...,X


In [29]:
with open('Subtask-A-master\\Subtask-A-master\\SubtaskA_Trial_Test_Labeled.csv','r',encoding='utf-8') as f:
    df_valid = pd.read_csv(f,names=['No','Text','Label'],delimiter=',')
    df_valid = df_valid.reset_index(drop=True)
    df_valid = df_valid.set_index('No')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe6 in position 678: invalid continuation byte